In [1]:
import pandas as pd
import backtrader as bt
import numpy as np
import io
import quandl
import csv
import datetime
import json

In [2]:
df = pd.read_csv('predicted.csv')
df['Open'] = df['SettlementPointPrice'].shift(1)
df['Close'] = df['SettlementPointPrice']
df['Date'] = pd.to_datetime(df['Date']).dt.to_pydatetime()
df = df[['Date', 'Open', 'Close', 'Predicted']]

In [12]:
class PandasData_Custom(bt.feeds.PandasData):
    lines = ('predicted',)
    params = (
        ('datetime', 0),
        ('open', 1),
        ('high', None),
        ('low', None),
        ('close', 2),
        ('volume', None),
        ('predicted', 3),
    )

data = PandasData_Custom(dataname=df)

In [13]:
class tradeStrategy(bt.Strategy):
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.predicted = self.datas[0].predicted
        
        self.order=None
        self.buyprice=None
        self.buycomm=None
        
        self.trades = io.StringIO()
        self.trades_writer = csv.writer(self.trades)
        
        self.operations = io.StringIO()
        self.operations_writer = csv.writer(self.operations)
        
        self.portfolioValue = io.StringIO()
        self.portfolioValue_writer = csv.writer(self.portfolioValue)
        
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.datetime(0)
        print("Datetime: {} Message: {} Predicted: {}".format(dt, txt, self.dataclose[0]))
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                ordertype = "BUY"
                #self.log("BUY EXECUTED, Price: {}, Cost: {}, Comm: {}".format(order.executed.price, order.executed.value, order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                ordertype = "SELL"
                #self.log("SELL EXECUTED, Price: {}, Cost: {}, Comm: {}".format(order.executed.price, order.executed.value, order.executed.comm))

            self.trades_writer.writerow([self.datas[0].datetime.datetime(0), ordertype, order.executed.price, order.executed.value, order.executed.comm])
        
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            #self.log("Order Canceled/Margin/Rejected")
            self.trades_writer.writerow([self.datas[0].datetime.datetime(0) , 'Rejection', 0, 0, 0])
            
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log('OPERATION PROFIT, GROSS: {}, NET: {}'.format(trade.pnl, trade.pnlcomm))
        self.operations_writer.writerow([self.datas[0].datetime.datetime(0), trade.pnlcomm])
    
    def get_logs(self):
        '''
        Returns:
        ________
        portfolioValue (df):
        Date and Value of portfolio
        
        trades (df):
        'Date', 'Type', 'Price', 'Total Spent', 'Comission'
        
        operations (df):
        'Date', 'Profit'
        '''
        self.portfolioValue.seek(0)
        portfolioValueDf = pd.read_csv(self.portfolioValue, names=['Date', 'Value'])
        
        portfolioValueDf['Date'] = pd.to_datetime(portfolioValueDf['Date'])
        portfolioValueDf = portfolioValueDf.set_index('Date')
        portfolioValueDf = portfolioValueDf.resample('1D').agg({'Date': lambda x: x.iloc[0], 'Value': lambda x: x.iloc[-1]})['Date']
        
        self.trades.seek(0)
        tradesDf = pd.read_csv(self.trades, names=['Date', 'Type', 'Price', 'Total Spent', 'Comission'])
        
        self.operations.seek(0)
        operationsDf = pd.read_csv(self.operations, names=['Date', 'Profit'])
        
        return portfolioValueDf, tradesDf, operationsDf
    
    
    def next(self):
        #self.log('Close: {}'.format(self.dataclose[0]))
        self.portfolioValue_writer.writerow([self.datas[0].datetime.datetime(0), self.broker.get_cash()])
        
        if self.order:
            return
        
        if not self.position:
            if self.predicted[0] > self.dataclose[0]:
                self.log("BUY CREATE {}".format(self.dataclose[0]))
                self.order = self.buy()
        else:
            if self.predicted[0] < self.dataclose[0]:
                self.log("SELL CREATE {}".format(self.dataclose[0]))
                self.order = self.sell()

In [14]:
cerebro = bt.Cerebro()

cerebro.adddata(data)
cerebro.addstrategy(tradeStrategy)
cerebro.addsizer(bt.sizers.SizerFix, stake=10)

cerebro.broker.setcash(10000)
cerebro.broker.setcommission(0.001)

print("Starting Value: {}".format(cerebro.broker.getvalue()))
run = cerebro.run()
print("Final Value: {}".format(cerebro.broker.getvalue()))

Starting Value: 10000


NameError: name 'order' is not defined

In [11]:
portfolioValue, trades, operations = run[0].get_logs()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:71: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version


In [13]:
portfolioValue.to_csv('portfolioValue.csv')
trades.to_csv('trades.csv')

# Space Here because another type

In [7]:
df = pd.read_csv('predicted.csv')
df = df.set_index('Date')

trades = pd.read_csv('trades.csv')
trades = trades.set_index('Date')

operations = pd.read_csv('operations.csv')
operations = operations.set_index('Date')

In [4]:
df['Buy'] = trades[trades['Type'] == 'BUY']['Price']
df['Sell'] = trades[trades['Type'] == 'SELL']['Price']

df['p/l'] = operations['Profit']
df['p/l'] = df['p/l'].replace(np.nan, 0)

df['Buy'] = df['Buy'].replace(np.nan, '')
df['Sell'] = df['Sell'].replace(np.nan, '')

In [5]:
df[['SettlementPointPrice', 'Buy', 'Sell', 'p/l']].to_csv('khuile.csv')

# Space here because another type

In [45]:
def sharpe_ratio(data):
    '''
    http://www.edge-fund.com/Lo02.pdf

    https://sixfigureinvesting.com/2013/09/daily-scaling-sharpe-sortino-excel/
    '''
    n = 365 ** 0.5 #365 trading days in crypto

    percentage = data.pct_change()[1:]
    sharpe = (np.average(percentage)/np.std(percentage)) * n
    return sharpe

def sortino_ratio(data):
    '''
    The ratio that only considers downside volatility
    '''
    n = 365 ** 0.5 #365 trading days in crypto
    df = pd.DataFrame(columns=['Portfolio', 'Percentage Change'])
    df['Portfolio'] = data
    df['Percentage Change'] = data.pct_change()
    df = df.fillna(method='bfill')
    negatives = df[df['Percentage Change'] < 0]['Percentage Change']
    sortino = (np.average(df['Percentage Change'])/np.std(negatives)) * n
    return sortino

def drawDown(down):
    '''
    Returns maximum draw down in terms of percentage
    '''

    minimum = ((np.amin(down) - down[0])/down[0]) * 100
    return minimum

def portfolio_return(data):
    return (data[-1] - data[0])/data[0]

# New Start

In [13]:
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='9CFF0BJ1ZPRQYEO0',output_format='pandas')
data, meta_data = ts.get_daily_adjusted('^GSPC')

In [28]:
spNew = data.reset_index()[['date', '5. adjusted close']].rename(columns={'date': 'Date', '5. adjusted close': 'Adj Close'})
spOld = pd.read_csv('S&P.csv')[['Date', 'Adj Close']]

In [37]:
spy = pd.concat([spOld, spNew[spNew['Date'] > spOld.iloc[-1]['Date']]]).reset_index(drop=True)

In [4]:
portfolioValue = pd.read_csv('portfolioValue.csv')
portfolioValue['Date'] = pd.to_datetime(portfolioValue['Date'])
portfolioValue = portfolioValue.set_index('Date')

In [47]:
portfolio_return(portfolioValue['s&pValue'])

1.383518427379833

In [ ]:
#get data from alphavantage and the csv to compare and for same timeline

In [41]:
sp = pd.read_csv('S&P.csv')[:-2][['Date', 'Adj Close']]
sp['Date'] = pd.to_datetime(sp['Date'])
sp = sp.set_index('Date')

In [42]:
portfolioValue['s&p'] = sp['Adj Close']
portfolioValue = portfolioValue.fillna(method='ffill')
portfolioValue['s&pValue'] = (portfolioValue['Value'].iloc[0] / portfolioValue['s&p'].iloc[0]) * portfolioValue['s&p']
portfolioValue = portfolioValue.drop('s&p', axis=1)

In [38]:
portfolioValue.to_csv('newPortfolioValue.csv')

In [44]:
sharpe = sharpe_ratio(portfolioValue['s&pValue'])

sortino = sortino_ratio(portfolioValue['s&pValue'])

drawdown = drawDown(portfolioValue['s&pValue'])

In [205]:
sharpe = sharpe_ratio(portfolioValue['Value'])
sortino = sortino_ratio(portfolioValue['Value'])
drawdown = drawDown(portfolioValue['Value'])

backtestMetrics = {'Sharpe Ratio': sharpe, 'Sortino Ratio': sortino, 'Maximum Drawdown': drawdown}

data = json.dumps(backtestMetrics)

with open("s&pMetrics.json", 'w') as fp:
    json.dump(data, fp)

In [207]:
sharpe = sharpe_ratio(portfolioValue['Value'])
sortino = sortino_ratio(portfolioValue['Value'])
drawdown = drawDown(portfolioValue['Value'])

backtestMetrics = {'Sharpe Ratio': sharpe, 'Sortino Ratio': sortino, 'Maximum Drawdown': drawdown}

data = json.dumps(backtestMetrics)

with open("strategyMetrics.json", 'w') as fp:
    json.dump(data, fp)

In [51]:
#Finally monthly returns and drawdown

In [200]:
def my_agg(x, column):
    names = {
        'Time': x.index[0].to_pydatetime().strftime("%B %Y"),
        'Return': (x[column].iloc[-1] - x[column].iloc[0])/x[column].iloc[0],
        'Drawdown': ((np.amin(x[column]) - x[column].iloc[0])/x[column].iloc[0])}

    return pd.Series(names, index=['Time', 'Return', 'Drawdown'])

In [208]:
portfolio = portfolioValue.resample('1M').apply(my_agg, column='Value').reset_index(drop=True)
portfolio.to_csv('strategyMovementDetails.csv', index=None)

In [204]:
spy = portfolioValue.resample('1M').apply(my_agg, column='s&pValue').reset_index(drop=True)
spy.to_csv('s&pMovementDetails.csv', index=None)